In [4]:
!pip -q install fasttext
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
import nltk
import string
import os
import numpy as np
import fasttext
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import brown
from nltk.corpus import wordnet
import io
import spacy
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
def readingfile(filename):
    with open(filename, 'r') as file:
        next(file)
        lines = file.readlines()
        # word1_list = []
        # word2_list = []
        # simlex999_list = []
        scores = {}

        for line in lines:
            data = line.strip().split('\t')
            # print(data)

            word1 = data[0]
            word2 = data[1]
            simlex999 = float(data[3])

            # word1_list.append(word1)
            # word2_list.append(word2)
            key = word1 + " " + word2
            scores[key] = simlex999
            # simlex999_list.append(simlex999)
    # for key in scores.keys():
    #     print(key, scores[key])
    return scores

def find_syns(word):
    syn = []
    ant = []
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            if '_' not in lemma.name():
                syn.append(lemma.name())
            if lemma.antonyms():
                if '_' not in lemma.antonyms()[0].name():
                    ant.append(lemma.antonyms()[0].name())

    syn = set(syn)
    ant = set(ant)
    return syn, ant

def cosine_similarity(array1, array2):
    dot_product = np.dot(array1, array2)
    norm_array1 = np.linalg.norm(array1)
    norm_array2 = np.linalg.norm(array2)
    cosine_similarity = dot_product / (norm_array1 * norm_array2)
    return cosine_similarity


def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [10]:
test_scores = readingfile("SimLex-999.txt")

tp, tn, fp, fn = 0, 0, 0, 0
decision_bound = 4
count = 0
boundary_score = 0.5

# if os.path.exists("model.bin"):
#     model = fasttext.load_model("model.bin")
# else:
#     model = fasttext.train_unsupervised("data.txt")
#     model.save_model("model.bin")

# filename = "wiki-news-300d-1M-subword.vec"

# data = load_vectors(filename)
# print(data)
# oldie = model.get_word_vector("Chomsky")
# age = model.get_word_vector("Antarctica")

# print(cosine_similarity(oldie, age))

for key in test_scores.keys():
    word1, word2 = key.split()
    score = test_scores[key]
    syn1, ant1 = find_syns(word1)
    syn2, ant2 = find_syns(word2)
    if (word1 in ant2) or (word2 in ant1):
        if (score < decision_bound):
            tn += 1
        else:
            fn += 1
        continue
            # print(word1, word2, "fn")
    elif (word1 in syn2) or (word2 in syn1):
        if (score >= decision_bound):
            tp += 1
        else:
            fp += 1
        continue
            # print(word1, word2, "fp")
    else:
        doc1 = nlp(word1)
        doc2 = nlp(word2)
        sim_score = doc1.similarity(doc2)
        if (sim_score > boundary_score):
            if (score > decision_bound):
                tp += 1
            else:
                fp += 1
        else:
            if (score > decision_bound):
                fn += 1
            else:
                tn += 1


        # print(word1, word2, sim_score)



if (tp + tn + fp + fn > 0):
    print(round((tp + tn)/(fp + fn + tp + tn), 3))
    # print(tp, tn, fp, fn)

0.649
336 312 132 219
